In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [6]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,sex From results_hard --where Date > '2023-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)
#data=data[data['Fav_Odds']>1.19]
#data=data[data['percent']>0.3]
#data=data[data['percent2']>0.1]

In [ ]:
data.to_csv('results.hard.csv')

In [ ]:
combine = pd.merge(todays_matches, data, how="left", left_on="Fav", right_on="Name")
    combine2 = pd.merge(combine, data, how="left", left_on="Dog", right_on="Name")
    combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
    combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )

In [ ]:
data1=data[(data['percent2']<0.6)&(data['percent']>0.6)&(data['Sex']=='Mens')]
len(data1[data1['Winner']==data1['Dog']])/len(data1)

0.29786036036036034

In [ ]:
print(len(data1))

1776


In [ ]:
data3=data[(data['Fav_Odds']>1.19)&(data['Fav_Odds']<1.3)&(data['percent2']<0.3)&(data['percent']>0.6)&(data['Sex']=='Mens')&(data['Dog_Games']>8)]
len(data3[data3['Winner']==data3['Fav']])/len(data3)

0.9642857142857143

In [ ]:
len(data3)

28

In [ ]:
data3[data3['Winner']!=data3['Fav']]

,Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,percent,percent2,Fav_Games,Dog_Games,Fav_Odds,Fav,Sex,Winx
3026,Liam Broady,Liam Broady,27%,4.39,80%,0.8,0.272727,10,11,1.21,Botic Van De Zandschulp,Mens,Dog
